<center> <h1> Heterogeneous Computing for AI </h1> </center>

<center> <h2> Lecture 02 -: Tutorial 01 </h2> </center>

<center> <h4> Raghava Mukkamala (rrm.digi@cbs.dk)</h4> </center>

Instructions

Please use Python 3 for working on the following questions.


## Example 01: Download URLs 

### The following python script downloads data from a set of webpages.
This is a classic example of downloading webpages and the standard for introducing concurrency.


In [11]:
import requests
import time

Utility functions to download the webpages

In [12]:
def download_site(url):
    
    with requests.get(url) as response:
        
        return len(response.content)


In [17]:
def download_all_sites(urls):
    """
        This function takes a list of URLs and call the 
        download_site()  function for each URL in the list.
    """
    for url in urls:
  
        print("For URL: ", url, download_site(url))


In [18]:
if __name__ == "__main__":
    
    start_time = time.time()

    links = ["https://twitter.com","https://facebook.com", "https://www.linkedin.com"] * 10
    download_all_sites(links)
    
    end_time = time.time()
    
    print("Time taken: ", end_time - start_time, "Seconds")


For URL:  https://twitter.com 125821
For URL:  https://facebook.com 69964
For URL:  https://www.linkedin.com 120506
For URL:  https://twitter.com 125176
For URL:  https://facebook.com 69962
For URL:  https://www.linkedin.com 120506
For URL:  https://twitter.com 125431
For URL:  https://facebook.com 69963
For URL:  https://www.linkedin.com 120506
For URL:  https://twitter.com 125200
For URL:  https://facebook.com 69966
For URL:  https://www.linkedin.com 120506
For URL:  https://twitter.com 125486
For URL:  https://facebook.com 69964
For URL:  https://www.linkedin.com 120506
For URL:  https://twitter.com 125456
For URL:  https://facebook.com 69966
For URL:  https://www.linkedin.com 120506
For URL:  https://twitter.com 125185
For URL:  https://facebook.com 69963
For URL:  https://www.linkedin.com 120506
For URL:  https://twitter.com 125187
For URL:  https://facebook.com 69960
For URL:  https://www.linkedin.com 120506
For URL:  https://twitter.com 125176
For URL:  https://facebook.com 6996

## Example 02:  Simple Threading 

A simple example showing how to create threads and assign functions to them

In [3]:
import threading
import time

In [5]:
def wait2_seconds():
    print(f"In function wait2_seconds, run by thread: {threading.current_thread().name}")

    time.sleep(2)
    
    print(f"Finished waiting 2 seconds")

In [6]:
def wait5_seconds():
    
    print(f"In function wait5_seconds, run by thread: {threading.current_thread().name}")
    
    time.sleep(5)
    
    print("Finished waiting 5 seconds")

In [11]:
if __name__ == "__main__":

    print(f"In main thread: ,run by thread: {threading.current_thread().name}")

    # creating threads
    t1 = threading.Thread(target=wait2_seconds, name='t1')
    
    t2 = threading.Thread(target=wait5_seconds, name='t2') 

    # start the threads
    t1.start()
    
    t2.start()

    # wait for threads to complete execution
    # This is done so that the "main thread" that is running all this code doesnt
    # terminate before the other threads finish their execution
    t1.join()
    
    t2.join()

    print("main thread is finished execution")



In main thread: ,run by thread: MainThread
In function wait2_seconds, run by thread: t1In function wait5_seconds, run by thread: t2

Finished waiting 2 seconds
Finished waiting 5 seconds
main thread is finished execution


## Example 03:  Threading Example 

Creating threads dynamically and timing them  

In [1]:
from time import sleep, perf_counter

from threading import Thread


In [2]:
def task(id):
    print(f'Starting the task {id}...')
    
    sleep(1)
    
    print(f'The task {id} completed \n')


Now we start the timer and create 10 threads to run the same function concurrently!

#### Especially run the following code and observe in what sequence threads started their execution and which sequence finished their execution. 

In [3]:
start_time = perf_counter()

# create and start 10 threads
threads = []

for n in range(1, 11):

    t = Thread(target=task, args=(n,))
    
    t.daemon = True
    
    threads.append(t)
    
    t.start()

# wait for the threads to complete
for t in threads:
    
    t.join()

end_time = perf_counter()

print(f'It took {end_time- start_time: 0.2f} second(s) to complete.')

Starting the task 1...
Starting the task 2...
Starting the task 3...
Starting the task 4...
Starting the task 5...
Starting the task 6...
Starting the task 7...Starting the task 8...

Starting the task 9...
Starting the task 10...
The task 2 completed 

The task 3 completed 
The task 1 completed 

The task 4 completed 


The task 6 completed 

The task 8 completed 

The task 5 completed 
The task 7 completed 
The task 9 completed 



The task 10 completed 

It took  1.01 second(s) to complete.


## Example 04:  A simple non-trivial parallel program
Source : Inspired from the examples in https://www.machinelearningplus.com/python/parallel-processing-python/

In [1]:
import time
from typing import List
import numpy as np 


In [2]:
def range_counter(row: List[int], min: int = 5, 
                  max: int = 10) -> int:
    '''
    Returns the number of values in the row that fall between the given range
        Args:
            i.   row : Lst of numbers
            ii.  min : minimum value of range
            iii. max : maximum values of range

        Returns: a count (int) of values that fall in the range
    '''
    count = 0
    
    for val in row:
        
        if min <= val <= max:
            count += 1
    
    return count


In [3]:
def apply_range_counter_sequential(data: List[List[int]],
                                   min: int,
                                   max: int) -> List[int]:
    '''
    This function takes data and applies the range_counter function
    over all the rows in the data
    '''
    result = []
    
    for row in data:

        temp_result = range_counter(row, min, max)

        result.append(temp_result)

    return result


In [4]:
if __name__ == '__main__':

    # Providing a seed ensures we can get the same 'random' values
    #  generated each time (Good for testing)
    np.random.seed(0)

    # create a matrix with dimensions 200x5 (200 rows and 5 columns)
    arr = np.random.randint(0, 10, size=[200, 10])

    # Convert into lists of lists
    data = arr.tolist()
    
    print(data[0:10])

    # timing the sequential solution
    start_sequential = time.perf_counter()

    seq_result = apply_range_counter_sequential(data, 5, 10)
    
    end_sequential = time.perf_counter()

    print(f'Finished Sequential computation in {round(end_sequential-start_sequential, 5)} seconds')

    print('First 10 results of the sequential result', seq_result[:10])

print('done!')


[[5, 0, 3, 3, 7, 9, 3, 5, 2, 4], [7, 6, 8, 8, 1, 6, 7, 7, 8, 1], [5, 9, 8, 9, 4, 3, 0, 3, 5, 0], [2, 3, 8, 1, 3, 3, 3, 7, 0, 1], [9, 9, 0, 4, 7, 3, 2, 7, 2, 0], [0, 4, 5, 5, 6, 8, 4, 1, 4, 9], [8, 1, 1, 7, 9, 9, 3, 6, 7, 2], [0, 3, 5, 9, 4, 4, 6, 4, 4, 3], [4, 4, 8, 4, 3, 7, 5, 5, 0, 1], [5, 9, 3, 0, 5, 0, 1, 2, 4, 2]]
Finished Sequential computation in 0.00012 seconds
First 10 results of the sequential result [4, 8, 5, 2, 4, 5, 6, 3, 4, 3]
done!


### To find number of processes/CPUs on your computer

In [1]:
import multiprocessing as mp

print("Number of processors: ", mp.cpu_count())

Number of processors:  16
